# TopAGNPS outlet identification

- This notebook executes the commands to:
    - Run the preprocessing step of TopAGNPS from presribed boundary
    - Identify the outlet
    - Finish the computation

In [9]:
import os, sys
sys.path.append('../..')
from glob import glob
import geopandas as gpd
from shapely.geometry.point import Point
import matplotlib.pyplot as plt

from src.pyagnps import topagnps
from src.pyagnps import utils


import importlib

In [2]:
CSA = 10 # Critical Source Area in ha
MSCL = 250 # Minimum Stream Channel Length in m
RESOLUTION = 30 # Resolution of the DEM in m
BUFFER = 500 # Buffer around the thuc in m

path_to_thucs = '/aims-nas/luc/data/tophuc_S_M_40000_closed_holes_with_container_thuc_merged_bbox_area_first_kept.gpkg'

root_dir = '/aims-nas/luc/thuc_runs_40k_SM/'
dir_runs_name = f'40000_SM_res_{RESOLUTION}_csa_{CSA}_mscl_{MSCL}_buff_{BUFFER}' # Name of the directory where the runs will be stored

thuc_id = '1139'
path_to_thuc = os.path.join(root_dir, dir_runs_name, f'thuc_{thuc_id}_{dir_runs_name}')

In [14]:
thucs = gpd.read_file(path_to_thucs)

In [12]:
files_in_thuc_dir = os.listdir(path_to_thuc)
files_in_thuc_dir

['thuc_1139_40000_SM_res_30_csa_10_mscl_250_buff_500_res_30_m.asc',
 'RELIEF_thuc_1139_40000_SM_res_30_csa_10_mscl_250_buff_500_res_30_m.asc',
 'UPAREA.PRJ',
 'NETFUL.ASC',
 'FLOVEC.ASC',
 'TopAGNPS_status.CSV',
 'valgrind_topagnps_preprocessing.txt',
 'TopAGNPS_log.CSV',
 'NETFUL.PRJ',
 'valgrind_topagnps_full_processing.txt',
 'FLOVEC.PRJ',
 'thuc_1139_40000_SM_res_30_csa_10_mscl_250_buff_500_res_30_m.prj',
 'TOPAGNPS.XML',
 'RELIEF_thuc_1139_40000_SM_res_30_csa_10_mscl_250_buff_500_res_30_m.prj',
 'UPAREA.ASC',
 'command_line_output.txt',
 'TopAGNPS_wrn.CSV']

In [15]:
thuc = thucs[thucs['tophucid']==thuc_id]
thuc

,tophucid,drainage_area_sqkm,outlethuc12,outlethuc12_hutype,receivinghuc12,contained_hucs,num_contained_hucs,foreign_inflows,num_foreign_inflows,tophucid_bbox_container,totophuc,bbox_area_sqkm,geometry
1138,1139,39494.29,051402060704,S,080101000103,"051302050807,051402040105,060400030506,0604000...",411,"051201141004,051402011204,060300051004,0603000...",11,None,1141,113827.728888,"POLYGON ((-88.51402 36.70802, -88.51484 36.707..."


In [11]:
files = {
    'uparea_asc': os.path.join(path_to_thuc, 'UPAREA.ASC'),
    'topagnpsxml' : os.path.join(path_to_thuc, 'TOPAGNPS.XML')
}

In [16]:
xout, yout, rowout, colout, raster_crs = topagnps.find_outlet_uparea_shape_intersection(files['uparea_asc'], thuc)
print(xout, yout, rowout, colout)
print(raster_crs)

309680.8126563854 4095283.453645436 4253 595
EPSG:32616


In [31]:
def rowcol2xy(row, col, nrows, xllcorner, yllcorner, cellsize):
    x = xllcorner + (col - 1)*cellsize + cellsize/2
    y = yllcorner + (nrows - row)*cellsize + cellsize/2
    return x, y

def xy2rowcol(x, y, nrows, xllcorner, yllcorner, cellsize):
    row = nrows - (y - yllcorner - cellsize/2)/cellsize
    col = 1 + (x - xllcorner - cellsize/2)/cellsize
    return row, col

nrows     = 12850
xllcorner = 291845.812656385417                                                   
yllcorner = 3837358.453645436093
cellsize  = 30.0


# xx, yy = rowcol2xy(4253, 595, nrows, xllcorner, yllcorner, cellsize)
xx, yy = rowcol2xy(743, 264, nrows, xllcorner, yllcorner, cellsize)
print(xx, yy)

299750.8126563854 4200583.453645436


In [18]:
thuc.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [20]:
outlet = gpd.GeoDataFrame(geometry=[Point(xout, yout)], crs=raster_crs)
outlet = outlet.to_crs(thuc.crs)

In [22]:
map = thuc.explore()
outlet.explore(m=map, color='red', marker_type='circle_marker', marker_kwds={'radius':8}, tooltip=False)